In [31]:
import math
import numpy as np

In [57]:
class Perceptron:
    def __init__(self,weights,bias):
        """
        inputs : a vector of inputs 
        weights : a vector of weights 
        output : the provided output
        """
        self.weights = np.array(weights)
        self.bias = bias
    
    def activation_function(self, x):
        # sigmoid for training
        return 1/(1+math.exp(-x))
    
    def Hard_activation_function(self, x):
        # step function for prediction (purely academic)
        return 1 if x > 0 else 0
    
    def predict(self,inputs):
        x = np.dot(self.weights, inputs) + self.bias
        return self.activation_function(x)

In [ ]:
class PerceptronLayer:
    def __init__(self, layer_id, n_inputs, n_neurons):
        self.layer_id = layer_id
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        
        # Vectorized weights and biases
        self.weights = np.random.randn(n_neurons, n_inputs) * 0.1
        self.biases = np.zeros((n_neurons, 1))
        
        # values for backprop
        self.z = None
        self.a = None
        self.inputs = None

    def activation(self, z):
        # Sigmoid
        return 1 / (1 + np.exp(-z))
    
    def activation_derivative(self, z):
        a = self.activation(z)
        return a * (1 - a)
    
    def forward(self, inputs):
        self.inputs = inputs
        self.z = np.dot(self.weights, inputs) + self.biases
        self.a = self.activation(self.z)
        return self.a

In [75]:
class FeedForwardNeuralNetwork:
    def __init__(self, n_inputs, n_outputs, hidden_layers):
        self.structure = [n_inputs] + hidden_layers + [n_outputs]
        self.layers = []
        
        for i in range(1, len(self.structure)):
            layer = PerceptronLayer(i-1, self.structure[i-1], self.structure[i])
            self.layers.append(layer)
    
    def forward(self, x):
        a = x
        for layer in self.layers:
            a = layer.forward(a)
        return a
    
    def backward(self, y_true, learning_rate=0.1):
        n_layers = len(self.layers)
        deltas = [None] * n_layers
        
        # output layer delta
        output_layer = self.layers[-1]
        deltas[-1] = (output_layer.a - y_true) * output_layer.activation_derivative(output_layer.z)
        
        # hidden layers delta
        for l in range(n_layers - 2, -1, -1):
            layer = self.layers[l]
            next_layer = self.layers[l+1]
            deltas[l] = np.dot(next_layer.weights.T, deltas[l+1]) * layer.activation_derivative(layer.z)
        
        # update weights and biases
        for l in range(n_layers):
            layer = self.layers[l]
            a_prev = self.layers[l-1].a if l > 0 else layer.inputs
            layer.weights -= learning_rate * np.dot(deltas[l], a_prev.T)
            layer.biases -= learning_rate * deltas[l]
    
    def train(self, X_train, Y_train, epochs=1000, lr=0.1):
        for epoch in range(epochs):
            loss = 0
            for x, y in zip(X_train, Y_train):
                x = np.array(x).reshape(-1,1)
                y = np.array(y).reshape(-1,1)
                output = self.forward(x)
                loss += np.sum((output - y)**2)/2
                self.backward(y, learning_rate=lr)
            if epoch % 500 == 0:
                print(f"Epoch {epoch}, Loss: {loss:.4f}")
    
    def predict(self, inputs):
        x = np.array(inputs).reshape(-1,1)
        output = self.forward(x)
        return output


In [76]:
net = FeedForwardNeuralNetwork(2, 1, [2])
X = [[0,0],[0,1],[1,0],[1,1]]
Y = [[0],[1],[1],[0]]
net.train(X, Y, epochs=5000, lr=0.5)

Epoch 0, Loss: 0.5245
Epoch 500, Loss: 0.5196
Epoch 1000, Loss: 0.5162
Epoch 1500, Loss: 0.4045
Epoch 2000, Loss: 0.1981
Epoch 2500, Loss: 0.0172
Epoch 3000, Loss: 0.0074
Epoch 3500, Loss: 0.0046
Epoch 4000, Loss: 0.0033
Epoch 4500, Loss: 0.0025
